In [ ]:
import LinearAlgebra

In [ ]:
import Random

In [ ]:
import BenchmarkTools

In [ ]:
import StaticArrays
# details on staticarrays here https://m3g.github.io/JuliaNotes.jl/stable/immutable/

In [ ]:
import NBInclude

In [ ]:
# import the code from the l,d motif simulation notebook
NBInclude.@nbinclude("simulate_ld_motif.ipynb")

# try calculating something like a gradient
which direction should we turn each dimension to increase the score?

In [ ]:
# it is written in such a way that it is continuously defined
function Score(Starts_inp, Seqs=sequences, Length=length(motif))
    maxScore = Length * length(Seqs)
        
    seqsMatrix = permutedims(reduce(hcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp)))
    
    # find the most common element in each column
    mostCommon = mapslices(StatsBase.mode, seqsMatrix, dims = 1)
    
    # now count how many sequences are equal to the consensus
    thisScore = sum(map((i, j) -> sum(i .== j), eachslice(seqsMatrix, dims = 2), mostCommon))
    
    return(maxScore - thisScore)
end

In [ ]:
# calculate a partial score in the plus and minus direction
# return positive if going in the positive direction increases the score
# return negative if going in the negative direction increases the score
# the value given is the sum score difference gained from going x-1 -> x -> x+1
function Grad(Starts_inp, seq)
    Starts_inp_mod_low = copy(Starts_inp)
    Starts_inp_mod_high = copy(Starts_inp)
    Starts_inp_mod_low[seq] -= 1
    Starts_inp_mod_high[seq] += 1
    score1 = Score(Starts_inp) - Score(Starts_inp_mod_low)
    score2 = Score(Starts_inp) - Score(Starts_inp_mod_high)
    return(score2 - score1)
end

In [ ]:
function CalcGrad(StartsInp)
    return Grad.((StartsInp,), 1:length(StartsInp))
end

In [ ]:
function Roll(Position, Velocity, tmax = 10)
    # copy the inputs so we don't modify them on accident
    position = copy(Position)
    
    velocity = copy(Velocity)
    score = zeros(Int64, tmax)
    # continue rolling until t > 10 or velocity is 0
    t = 1
    while (t <= tmax) & (any(velocity .!= 0))
        # calculate the current score
        score[t] = Score(position)
        
        println(" ==== t ==== ", t)
        println("pos: ",  position)
        println("vel: ", velocity)
        println("sco: ", score[t])
        println("grd: ", CalcGrad(position))
        println()
        
        # "integrate" over the scores in the path of the particle
        # only move in discrete steps, though
        # will need to add leap-frog steps in discrete space later, though
        
        # strategy here will be to ALWAYS move in the direction of the maximum gradient,
        # and then sometimes move in the other directions, randomly
        move_prob = abs.(float.(CalcGrad(position)))
        move_prob ./= maximum(move_prob)
        move_accep = rand(length(velocity)) .<= move_prob
        # update the position using the velocity - go in the direction of the velocity
        position .+= sign.(velocity) .* move_accep
        
        # shrink the velocity by one in each dimension to simulate friction
        velocity .-= CalcGrad(position) .* move_accep
        t += 1
    end
end

In [ ]:
# set the seed
Random.seed!(100)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 14, 100, 0)

In [ ]:
Random.seed!(100)
Roll([2,2,2], [3, 3, 3], 5)

In [ ]:
#BenchmarkTools.@btime Roll([54, 26, 6], [3, 3, 3], 20)